<a href="https://colab.research.google.com/github/Vampaxx/Pyspark_basics/blob/main/create_dataframe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=8fad9c5a71b19b7306e5504c520b5bc6109c0e24bc9ebb16973ed2c355a0e5ab
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from pyspark.sql import SparkSession
import numpy as np
from pyspark.sql.functions import col,isnan, when, count

In [4]:
csv_file = ('/content/drive/MyDrive/Pyspark/datas/Clean_Dataset.csv')

In [5]:
# create session
spark = SparkSession.builder.appName('Missing_value').getOrCreate()

In [6]:
df = spark.read.csv(csv_file,header=True,inferSchema=True)

## Another Methods

In [7]:
spark.read.format('csv').load(csv_file).show()


+----+---------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
| _c0|      _c1|    _c2|        _c3|           _c4|  _c5|          _c6|             _c7|    _c8|     _c9|     _c10| _c11|
+----+---------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|NULL|  airline| flight|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|price|
|   0| SpiceJet|SG-8709|      Delhi|       Evening| zero|        Night|          Mumbai|Economy|    2.17|        1| 5953|
|   1| SpiceJet|SG-8157|      Delhi| Early_Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5953|
|   2|  AirAsia| I5-764|      Delhi| Early_Morning| zero|Early_Morning|          Mumbai|Economy|    2.17|        1| 5956|
|   3|  Vistara| UK-995|      Delhi|       Morning| zero|    Afternoon|          Mumbai|Economy|    2.25|        1| 5955|
|   4|  Vistara| UK-963|

In [8]:
# read with header
spark.read.option("header",True)\
.option("inferSchema", "true")\
.csv(csv_file).show()

+---+---------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|_c0|  airline| flight|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|price|
+---+---------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|  0| SpiceJet|SG-8709|      Delhi|       Evening| zero|        Night|          Mumbai|Economy|    2.17|        1| 5953|
|  1| SpiceJet|SG-8157|      Delhi| Early_Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5953|
|  2|  AirAsia| I5-764|      Delhi| Early_Morning| zero|Early_Morning|          Mumbai|Economy|    2.17|        1| 5956|
|  3|  Vistara| UK-995|      Delhi|       Morning| zero|    Afternoon|          Mumbai|Economy|    2.25|        1| 5955|
|  4|  Vistara| UK-963|      Delhi|       Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5955|
|  5|  Vistara| UK-945|      Del

In [9]:
# read with header
spark.read.option("header",True)\
.option("inferSchema", "true")\
.csv(csv_file).withColumnRenamed("_c0",'Id').show()

+---+---------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
| Id|  airline| flight|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|price|
+---+---------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|  0| SpiceJet|SG-8709|      Delhi|       Evening| zero|        Night|          Mumbai|Economy|    2.17|        1| 5953|
|  1| SpiceJet|SG-8157|      Delhi| Early_Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5953|
|  2|  AirAsia| I5-764|      Delhi| Early_Morning| zero|Early_Morning|          Mumbai|Economy|    2.17|        1| 5956|
|  3|  Vistara| UK-995|      Delhi|       Morning| zero|    Afternoon|          Mumbai|Economy|    2.25|        1| 5955|
|  4|  Vistara| UK-963|      Delhi|       Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5955|
|  5|  Vistara| UK-945|      Del

## Method 1

In [10]:
data = [(1,'arjun'),
        (2,'micheal'),
        (3,'mia'),
        (4,'jackson'),
        (5,'khalifa')]
schema = ['Emp_no','Name']

In [11]:
df = spark.createDataFrame(data,schema)
df.show()
df.printSchema()

+------+-------+
|Emp_no|   Name|
+------+-------+
|     1|  arjun|
|     2|micheal|
|     3|    mia|
|     4|jackson|
|     5|khalifa|
+------+-------+

root
 |-- Emp_no: long (nullable = true)
 |-- Name: string (nullable = true)



## Method 2

In [12]:
data = [{'id':1, 'Name':'Arjun'},
        {'id':2, 'Name':'Micheal'}]
df = spark.createDataFrame(data)
df.show()
df.printSchema()

+-------+---+
|   Name| id|
+-------+---+
|  Arjun|  1|
|Micheal|  2|
+-------+---+

root
 |-- Name: string (nullable = true)
 |-- id: long (nullable = true)



## Method 3

In [13]:
## Create a DataFrame with the explicit schema specified.

from pyspark.sql.types import *

In [14]:
data = [(1,'arjun'),
        (2,'micheal'),
        (3,'mia'),
        (4,'jackson'),
        (5,'khalifa')]
schema = StructType([
    StructField("Emp_no", IntegerType(), True),
    StructField("Name",   StringType(), True)])

In [15]:
spark.createDataFrame(data,schema).printSchema()

root
 |-- Emp_no: integer (nullable = true)
 |-- Name: string (nullable = true)



In [16]:
spark.createDataFrame(data,schema).collect()

[Row(Emp_no=1, Name='arjun'),
 Row(Emp_no=2, Name='micheal'),
 Row(Emp_no=3, Name='mia'),
 Row(Emp_no=4, Name='jackson'),
 Row(Emp_no=5, Name='khalifa')]

In [17]:
spark.read.format('csv')\
.option(key='header',value=True)\
.load(csv_file)\
.withColumnRenamed('_c0','Id').show(5)


+---+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
| Id| airline| flight|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|price|
+---+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|  0|SpiceJet|SG-8709|      Delhi|       Evening| zero|        Night|          Mumbai|Economy|    2.17|        1| 5953|
|  1|SpiceJet|SG-8157|      Delhi| Early_Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5953|
|  2| AirAsia| I5-764|      Delhi| Early_Morning| zero|Early_Morning|          Mumbai|Economy|    2.17|        1| 5956|
|  3| Vistara| UK-995|      Delhi|       Morning| zero|    Afternoon|          Mumbai|Economy|    2.25|        1| 5955|
|  4| Vistara| UK-963|      Delhi|       Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5955|
+---+--------+-------+-----------+------

## read multiple csv files from different folders and if inferschema is same for both files
file_path = [path_1,path_2]

spark.read.csv(file_path,header=True).show()

## if all csv files are in same folder then we use
file_path = 'folder/folder_of_files/'

spark.read.csv(file_path,header=True).show()

In [18]:
# read with header
df = spark.read.option("header",True)\
    .option("inferSchema", "true")\
    .csv(csv_file).withColumnRenamed("_c0",'Id')

In [19]:
df = spark.read.csv(csv_file,header=True,inferSchema=False)

In [20]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- flight: string (nullable = true)
 |-- source_city: string (nullable = true)
 |-- departure_time: string (nullable = true)
 |-- stops: string (nullable = true)
 |-- arrival_time: string (nullable = true)
 |-- destination_city: string (nullable = true)
 |-- class: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- days_left: string (nullable = true)
 |-- price: string (nullable = true)



In [21]:
categorical_features = [col for col,data_type in df.dtypes if data_type =='string']
numerical_features   = [col for col,data_type in df.dtypes if data_type == 'int']

In [22]:
[col for col,data_type in df.dtypes if data_type == 'int']

[]

In [23]:
#here every column is string type, so we need to change the data type
from pyspark.sql.types import *

In [24]:
schema = StructType().add(field='Id',data_type = IntegerType())\
                     .add(field='airline',data_type = StringType())\
                     .add(field='flight',data_type = StringType())\
                     .add(field='source_city',data_type = StringType())\
                     .add(field='departure_time',data_type = StringType())\
                     .add(field='stops',data_type = StringType())\
                     .add(field='arrival_time',data_type = StringType())\
                     .add(field='destination_city',data_type = StringType())\
                     .add(field='class',data_type = StringType())\
                     .add(field='days_left',data_type = IntegerType())\
                     .add(field='price',data_type = IntegerType())

In [34]:
df = spark.read.csv(csv_file,schema=schema,header=True)

In [26]:
df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- airline: string (nullable = true)
 |-- flight: string (nullable = true)
 |-- source_city: string (nullable = true)
 |-- departure_time: string (nullable = true)
 |-- stops: string (nullable = true)
 |-- arrival_time: string (nullable = true)
 |-- destination_city: string (nullable = true)
 |-- class: string (nullable = true)
 |-- days_left: integer (nullable = true)
 |-- price: integer (nullable = true)



In [27]:
new_field = []

for col in df.columns:

  if col in categorical_features:
    new_field.append(StructField(name=col,dataType=StringType()))

  if col in numerical_features:
    new_field.append(StructField(name=col,dataType=IntegerType()))


new_schema = StructType(new_field)
spark.read.csv(csv_file,schema=new_schema,header=True).printSchema()

root
 |-- airline: string (nullable = true)
 |-- flight: string (nullable = true)
 |-- source_city: string (nullable = true)
 |-- departure_time: string (nullable = true)
 |-- stops: string (nullable = true)
 |-- arrival_time: string (nullable = true)
 |-- destination_city: string (nullable = true)
 |-- class: string (nullable = true)
 |-- days_left: string (nullable = true)
 |-- price: string (nullable = true)



In [35]:
# df.write.parquet("airline_parquet_file.parquet")

## Show()